#### Secciones

+ Agrupamiento de datos
+ Union de dataframes
+ Visualizaciones

En uno de los ejercicios vistos en clase teniamos dos listas pareadas con los pesos moleculares de los aminoácidos.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/Ferorti/escuela2021/main/data/america.csv")

In [ ]:
df['letalidad'] = df['fallecidos'] / df['casos_totales'] * 100

Muchas veces tenemos que agrupar datos según más de una categoría.

Con lo que aprendimos podríamos realizar subsets o querys para generar distintos dataframes con cada categoría, por ej:


In [ ]:
sudamerica = df.query('region == "S"')
centroamerica = df.query('region == "C"')
norteamerica = df.query('region == "N"')

Esto es un proceso poco practico y menos aún si hay muchas categorías. 

Con pandas podemos agrupar dataframes automaticamente por una columna categórica utilizando el método ```.groupby()```

In [ ]:
dfs = {}
for region, grupo in df.groupby('region'):
    dfs[region] = grupo

In [ ]:
dfs['S']

Además de crear datafrmes, el método groupby nos permite realizar directamente operaciones para cada grupo:

In [ ]:
df.groupby('region').sum()

In [ ]:
df.groupby('region').mean().round()

In [ ]:
df.groupby('region').fallecidos.sum()

Utilizando el método .agg() podemos aplicar varias funciones sobre los grupos:

In [ ]:
df.groupby('region').fallecidos.agg(["sum",'count', 'mean', 'std']).round()

In [ ]:
df.groupby('region').agg({'fallecidos':['sum', 'mean'], 'letalidad':['mean', 'max', 'min']}).round(2)

In [ ]:
def listar_paises(lista):
    return ", ".join(lista)
    

In [ ]:
df.groupby('region').agg({'fallecidos':['sum', 'mean'], 'letalidad':['mean', 'max', 'min'], 'pais':['count', listar_paises]}).round(2)

Combinar DataFrames

En el caso de que tengamos diferentes tablas, combinarlas de diferentes formas:

+ podemos concatener dataframes con la función ```pd.concat()``` tanto con filas como con columnas:

+ podemos unir distintos datafarme utilizando el método ```.merge()``` o el método ```.join()```

Juegmos un poco con el dataframe de los pesos moleculares

In [ ]:
pms = [['I', 'Ile', 131.1736], ['L', 'Leu', 131.1736], ['K', 'Lys', 146.1882], ['M', 'Met', 149.2124], ['F', 'Phe', 165.19], ['T', 'Thr', 119.1197], ['W', 'Trp', 204.2262], ['V', 'Val', 117.1469], ['R', 'Arg', 174.2017], ['H', 'His', 155.1552], ['A', 'Ala', 89.0935], ['N', 'Asn', 132.1184],
 ['D', 'Asp', 133.1032], ['C', 'Cys', 121.159], ['E', 'Glu', 147.1299], ['Q', 'Gln', 146.1451], ['G', 'Gly', 75.0669], ['P', 'Pro', 115.131], ['S', 'Ser', 105.093], ['Y', 'Tyr', 181.1894]]

pesos_moleculares = pd.DataFrame(pms, columns=['aa', 'codigo', 'pesos'])

In [ ]:
primeros = pesos_moleculares.head()
primeros

In [ ]:
ultimos = pesos_moleculares.tail()
ultimos

In [ ]:
codigos = pesos_moleculares[['aa', 'codigo']]
codigos

In [ ]:
pesos = pesos_moleculares[['aa','pesos']]
pesos

In [ ]:
# Concatener uno abajo del otro
pd.concat([primeros, ultimos])

In [ ]:
pd.concat([codigos, pesos], axis=1).T.drop_duplicates().T

Qué pasa si estan mezclados?

In [ ]:
pesos_reordenados = pesos.sort_values('aa').reset_index(drop=True)
pesos_reordenados

In [ ]:
pd.concat([codigos,pesos_reordenados], axis=1)

In [ ]:
codigos.merge(pesos_reordenados)

Dependiendo de el parametro how podemos quedaron con distintos resultados:

![](https://www.softwaretestinghelp.com/wp-content/qa/uploads/2019/05/Capture-1.jpg)

In [ ]:
codigos.merge(pesos_reordenados.head()) # ver la ayuda para distintas combinaciones 

join funciona similar pero por indices:


In [ ]:
codigos.join(pesos_reordenados.set_index('aa'), 'aa')

Truquito

In [ ]:
from google.colab.data_table import DataTable as View
import pandas as pd
uniprots = pd.read_csv("https://raw.githubusercontent.com/Ferorti/escuela2021/main/data/swiss_prot.tab")
View(uniprots)

#### Visualizaciones

Pandas tiene el método ```.plot()``` para realizar gráficos simples sobre dataframes y series.

A continuación vamos a ver algunos ejemplos:

In [ ]:
df = df.set_index("pais")

+ Barras

In [ ]:
df.letalidad.plot.bar() # desordenado

In [ ]:
df.letalidad.sort_values().plot.bar()

In [ ]:
df.letalidad.sort_values(ascending=False).plot.bar()

In [ ]:
df.letalidad.sort_values().plot.barh()

In [ ]:
df_coloreado = df.sort_values('letalidad')
df_coloreado['color'] = df.region.map({'N':'steelblue','C':'seagreen','S':'lightcoral'})

In [ ]:
df_coloreado.sort_values('letalidad').plot.barh(y='letalidad', color=df_coloreado.color, figsize=(6,6), title='Letalidad por pais', legend=None)

In [ ]:
df_plot = df.groupby('region').sum()[['casos_totales','recuperados']]
display(df_plot)

df_plot.plot.bar()

In [ ]:
df_plot.plot.bar(stacked=True)

* Pieplots

In [ ]:
df.region.value_counts().plot.pie()

In [ ]:
df.sort_values('casos_totales').head(12).plot.scatter(x='casos_totales', y = 'fallecidos')

* boxplots

In [ ]:
iris = pd.read_csv("https://raw.githubusercontent.com/Ferorti/escuela2021/main/data/iris.data.csv")

In [ ]:
iris.boxplot(by='variedad', figsize=(10,10));

In [ ]:
iris.boxplot(column='petal_length', by='variedad', grid=False, fontsize=14);

In [ ]:
import seaborn as sns

In [ ]:
sns.boxplot(data=iris, x='petal_length', y='variedad')

In [ ]:
for g,grupo in iris.groupby('variedad'):
    sns.distplot(grupo.sepal_length, label=g)

In [ ]:
for g,grupo in iris.groupby('variedad'):
    sns.distplot(grupo.sepal_length, label=g, hist=None)

In [ ]:
sns.scatterplot(data=iris, x='sepal_length', y ='petal_length')

In [ ]:
sns.scatterplot(data=iris, x='sepal_length', y ='petal_length', hue='variedad')

In [ ]:
sns.pairplot(iris, hue='variedad')